### Variáveis

In [ ]:
numero_de_imoveis = 50
desconto = 30
modalidades = ["1º Leilão SFI","2º Leilão SFI","Licitação Aberta","Licitação Fechada", "Leilão SFI - Edital Único"]#, "Venda Online", "Venda Direta Online"
tipos_imovel = ["Casa","Apartamento","Sobrado","Terreno"]

path = "/content/drive/MyDrive/Imóveis Caixa/"
#URL da planilha completa
URL = "https://venda-imoveis.caixa.gov.br/listaweb/Lista_imoveis_SP.csv?115780016"

### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
useragents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',]

In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install libu2f-udev libvulkan1
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!wget https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip
!unzip -j chromedriver-linux64.zip chromedriver-linux64/chromedriver -d /usr/local/bin/
!pip install selenium-stealth
!pip install webdriver-manager
# !pip install selenium-requests

In [ ]:
import pandas as pd
import numpy as np
import io
import re
import time
import IPython
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# from seleniumrequests import Chrome
from selenium_stealth import stealth

## **Importação dos dados**

In [ ]:
def parse_cookies(cookies):
  cookie_str = ''
  for cookie in cookies.items():
    cookie_str = cookie_str + cookie[0] + '=' + cookie[1] + '; '
  return cookie_str

cookie_str = parse_cookies(pickle.load(open(path + "cookies.pkl", "rb")))

In [ ]:
header={
    "User-Agent": useragents[np.random.randint(0,len(useragents))],
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection':	'keep-alive',
    # 'Cookie': '__uzmc=413817382641; __uzmd=1757881949; SIMOV2=ffffffff09f69e3845525d5f4f58455e445a4a423660;', # cookies manuais
    # 'Cookie': cookie_str,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/jxl,image/webp,*/*;q=0.8'
}

session = requests.Session()
r = session.get(URL, headers=header)

In [ ]:
pickle.dump(session.cookies.get_dict(), open(path + "cookies.pkl", "wb"))

#checar se o bot foi detectado, se sim, tentar com cookies novos
if re.search('Radware Bot Manager',r.content.decode('latin-1')) != None:
  print('bot detectado')
  header['Cookie'] = cookie_str
  r = requests.get(URL, headers=header)

In [ ]:
s = r.content

data_preco = pd.read_csv(io.StringIO(s.decode('latin-1')), sep=';', skiprows=4, usecols=[5], names=["preco"], encoding='latin-1')
data_preco = data_preco.dropna()
identidade = data_preco["preco"].str.replace('.','', regex=False).str.split(',', regex=False).str[0].str.isdigit()

complemento = data_preco[~identidade]
if len(complemento) > 0:
  if complemento.iloc[0][0].isdigit():
    raise Exception("Complemento de endereço não processado. Rever código")

In [ ]:
skipped_rows = [0,1,2,3]
complemento_index = [x+4 for x in list(complemento.index)]
skipped_rows.extend(complemento_index)
print(skipped_rows)
colnames=["numero","UF","cidade","bairro","endereco","preco","avaliacao","desconto", "descricao","modalidade","link"]
data = pd.read_csv(io.StringIO(s.decode('latin-1')), sep=';', skiprows=skipped_rows, names=colnames, index_col=False, encoding='latin-1')

In [ ]:
colnames.insert(5, "complemento")
print(complemento_index)
data_complemento = pd.read_csv(URL, sep=';', skiprows=lambda x: x not in complemento_index, names=colnames, index_col=False, encoding='latin-1')
data.insert(5, "complemento", np.nan)
data = pd.concat([data,data_complemento])

In [ ]:
data["preco"] = data["preco"].str.replace('.','', regex=False).str.replace(',', '.', regex=False)
data["avaliacao"] = data["avaliacao"].str.replace('.','', regex=False).str.replace(',', '.', regex=False)
data = data.astype({"preco": "float64", "avaliacao": "float64"})

# **Tratamento**

### Filtros

In [ ]:
#tipo de imóvel
data = data.loc[data["descricao"].str.split(',').str[0].isin(tipos_imovel)]

#modalidade de venda
data = data.loc[data["modalidade"].isin(modalidades)]

#por desconto
data = data.loc[(data["desconto"] >= desconto) | (data["desconto"] == 0)]

#por preço
#data = data.loc[data["preco"] <= 600000]

### Seleção de dados novos

In [ ]:
complete_path = path + "backlog.csv"
data_old = pd.read_csv(complete_path, sep=';', decimal=',', index_col=False, encoding='utf-8')

In [ ]:
mask = data.merge(data_old, how="inner", on=["numero","desconto"])
data_new = data[~data["numero"].isin(mask["numero"])]

#primeira vez que o imóvel é postado?
data_new.insert(11,"primeiro","S")
data_new.loc[ data_new["numero"].isin(data_old["numero"]) , "primeiro"] = "N"


In [ ]:
data_new.set_index(data_new["numero"], inplace=True)
data_new = data_new.drop(columns=["numero"])

In [ ]:
#remover incompatíveis
delete = pd.read_csv(path + "invalidos.csv", names=["invalidos"])

In [ ]:
data_new = data_new[~data_new.index.isin(delete.invalidos)]
data_new = data_new.sort_values(by=["primeiro", "desconto"], ascending=[False, False]).head(numero_de_imoveis)

### Formatação dos valores

In [ ]:
data_new['text_content'] = ''
data_new['area'] = 0.0
data_new['area'] = data_new['area'].astype(float)
data_new['financiamento'] = ''

In [ ]:
data_new.descricao = data_new.descricao.str.split(pat=',', n=0).str[0]

### Scrapping

In [ ]:
def start_wd():
  service = ChromeService(executable_path=ChromeDriverManager().install())

  options = webdriver.ChromeOptions()
  options.add_argument("--headless")
  options.add_argument('--disable-blink-features=AutomationControlled')
  options.add_argument('--disable-popup-blocking')
  options.add_argument('--start-maximized')
  options.add_argument('--disable-extensions')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  wd = Chrome(service=service, options=options)
  wd.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")


  wd.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent":useragents[np.random.randint(0,len(useragents))]})


  stealth(wd,
        languages=["pt-BR", "en-US"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )

  return wd

In [ ]:
def quit_wd(wd):
  pickle.dump(wd.get_cookies(), open(path + "cookies.pkl", "wb"))
  cookie = ''
  for c in wd.get_cookies():
      cookie += '{name}={value}; '.format(
          name=c['name'],
          value=c['value']
      )
  wd.close()
  wd.quit()

In [ ]:
def coleta(i):
  #seleciona conteúdos da caixa
  content = wd.find_elements(By.CSS_SELECTOR, "div.content-wrapper div:nth-child(-n+3) h5, div.content-wrapper div:nth-child(-n+3) span, div.content-wrapper div:nth-child(-n+3) p")#, div.content-wrapper div:nth-child(-n+3) br
  #content = wd.find_elements(By.CSS_SELECTOR, "div.content-wrapper div:nth-child(-n+3)")
  #salva imagem no cache
  picpath = path + "fotos/" + str(i) +".png"
  try:
    with open(picpath, "wb") as file:
      preview = wd.find_element(By.CSS_SELECTOR, "img#preview")
      file.write(preview.screenshot_as_png)
      #print("foto salva")
  except:
    print("sem foto")

  content.pop(2)
  content.pop(11)
  text_content = "<h3>" + content[0].get_attribute("innerHTML") + "</h3><br><br>" + "<h3>" + re.sub(r"<br>", "</h3><h3>",content[1].get_attribute("innerHTML")) + "</h3><br><br>"
  for tag in content[2:]:
    text_content += "<p>" + tag.get_attribute("innerHTML") + "</p>"
    #print(tag.get_attribute("innerHTML"))

  #prepara texto
  text_content = re.sub(r"\t", "", text_content)
  text_content = re.sub(r"\n", "", text_content)
  text_content = re.sub(r"'", "", text_content)
  text_content = re.sub("span>", "p>", text_content)
  text_content = re.sub("<input.*?>", "", text_content)
  text_content = re.sub("<i.*?</i>", "", text_content)
  text_content = re.sub("<a.*?</a>", "", text_content)

  data_new.loc[i, "text_content"] = text_content

In [ ]:
def coleta_características(i):
  imovel = data_new.loc[data_new.index == i].iloc[0]
  if imovel.descricao != 'Terreno':
    area = re.search(r'(?<=Área privativa = <strong>)(\d|,)*(?=m2)', imovel.text_content)
    if area != None:
      area = area.group(0)
    else:
      area = re.search(r'(?<=Área total = <strong>)(\d|,)*(?=m2)', imovel.text_content)
      if area != None:
        area = area.group(0)
  else:
    area = re.search(r'(?<=Área do terreno = <strong>)(\d|,)*(?=m2)', imovel.text_content)
    if area != None:
      area = area.group(0)


  financiamento = 'à vista' if re.search('(NÃO aceita financiamento)', imovel.text_content) else 'financiamento'

  data_new.loc[i, 'financiamento'] = financiamento
  if area == None: return

  area = re.sub(r"\.","",area)
  area = re.sub(r",",".",area)

  data_new.loc[i, 'area'] = float(area)

In [ ]:
delete = []
CEP_list = []
date_list = []
nao_encontrados =[]

In [ ]:
strike = 0
# wd = start_wd()

for i, imovel in data_new.iterrows():
  url = imovel.link
  to = np.random.randint(5,12) #timeout
  wd.set_page_load_timeout(to)
  try:
    wd.get(url)
    #wd.uc_open(url)
  except:
    print(f"timeout {to}")
    nao_encontrados.append(i)
    continue
  print(i)
  wd.execute_async_script( #scroll pra baixo
            """
        count = 400;
        let callback = arguments[arguments.length - 1];
        t = setTimeout(function scrolldown(){
            console.log(count, t);
            window.scrollTo(0, count);
            if(count < (document.body.scrollHeight || document.documentElement.scrollHeight)){
              count+= 400;
              t = setTimeout(scrolldown, 1000);
            }else{
              callback((document.body.scrollHeight || document.documentElement.scrollHeight));
            }
        }, 1000);"""
        )
  try:
    judiciaria = wd.find_element(By.CSS_SELECTOR, "div.related-box p:last-of-type")
    judiciaria = judiciaria.get_attribute("innerHTML").find("judicial")
  except:
    judiciaria = -1
  if judiciaria != -1:
    delete.append(i)
    print("acao judiciaria")
    # imóveis com ação judiciária não são desejados
    continue
  if imovel.modalidade == "Leilão SFI - Edital Único":
    pos_date = "5"
    if imovel.desconto == "0,0":
      try:
        valor_leilao = wd.find_element(By.CSS_SELECTOR, "div.content > p").get_attribute("innerHTML")
        valor_leilao = re.search('%s(.*)' % (r"Valor mínimo de venda 2º Leilão: R\$ "), valor_leilao).group(1)
        valor_leilao = re.sub(r"\.", "", valor_leilao)
        valor_leilao = re.sub(r"</b>", "", valor_leilao)
        print(valor_leilao)
        data_new.loc[i, "preco"] = valor_leilao
      except:
        print("valor 2o leilao nao encontrado")
        nao_encontrados.append(i)
        continue
  elif imovel.modalidade == "Licitação Aberta":
    pos_date = "4"
  else:
    CEP_list.append("")
    date_list.append("")
    continue
  try:
    date = wd.find_element(By.CSS_SELECTOR, "div.related-box span:nth-of-type(" + pos_date + ")")
    date = date.text.split(' - ', 2)[1]
    address = wd.find_element(By.CSS_SELECTOR, "div.related-box p:nth-of-type(1)")
    CEP = re.search('%s(.*)%s' % ("CEP: ", ","), address.text).group(1)
    CEP_list.append(CEP)
    date_list.append(date)
  except:
    delete.append(i)
    print("dados nao encontrados")
    print(re.search('comportamento malicioso', wd.page_source) != None)
    if re.search('comportamento malicioso', wd.page_source) != None:
      print("bot identificados")
      quit_wd(wd)
      wd = start_wd()
    strike += 1
    time.sleep(np.random.randint(3,5))
    if strike == 2:
      break
    time.sleep(np.random.randint(3,5))
    continue
  coleta(i)

  coleta_características(i)

  time.sleep(np.random.randint(3,5))

In [ ]:
adiados = list(set(nao_encontrados) - set(delete))
data_new.drop(index=adiados, inplace=True)

In [ ]:
data_new.drop(index=delete, inplace=True)
if len(CEP_list) < len(data_new):
  data_new = data_new.head(len(CEP_list))
data_new.insert(4,"CEP",CEP_list)
data_new.insert(9,"data",date_list)

In [ ]:
# volta os precos ao valor original se o desconto do 2o leilao for menor que o desconto desejado
# assim são salvos na lista completa e não são considerados até mudarem de preco
data_new.loc[(data_new.preco/data_new.avaliacao) > (1-desconto*.01), "preco"] = data.loc[data.numero.isin(data_new.loc[(data_new.preco/data_new.avaliacao) > (1-desconto*.01)].index), "preco"]

### Salvamento

In [ ]:
pd.DataFrame(delete).to_csv(path + "invalidos.csv",sep = ';', mode='a', header=False, index=False)

Atualização da planilha completa

In [ ]:
data_new[['desconto','data']].to_csv(complete_path, mode='a', index=True, index_label="numero", sep=';', decimal=',', header=False)

In [ ]:
# reorganização das colunas
data_new = data_new[['UF', 'cidade', 'bairro', 'CEP', 'financiamento', 'data', 'avaliacao', 'preco', 'descricao', 'link', 'area', 'text_content']]

Criação da planilha de novos imóveis

In [ ]:
output_path = path + "imoveis_diarios.csv"
import csv
f = open(output_path, 'w')
writer = csv.writer(f, delimiter=';')
cond = ['']*11
cond.insert(8,str(desconto) + "% ou mais")
cond.insert(10,' '.join(modalidades))
writer.writerow(cond)
writer.writerow(['']*13)
header = ["N° do imóvel","UF","Cidade","Bairro", "CEP", "Financiamento", "Data", "Valor de avaliação", "Preço", "Descrição", "Link de acesso", "Area", "text_content"]
writer.writerow(header)
f.close()

Salvamento dos dados

In [ ]:
data_new.to_csv(output_path, mode='a', index=True, index_label="N° do imóvel", sep=';', decimal=',', header=False, encoding='utf-8')